# Noisy and Repellant

Your task with this assignment is to parallelize a serial application using OpenMP, and to use profiling tools to assess different approaches to that parallelization.

**Due Thursday, September 19, before class**

This assignment is intended for the CPU-only nodes, where we can get a lot of CPU thread concurrency.

In [ ]:
module use $CSE6230_DIR/modulefiles
module load cse6230

Last time, we modelled non-interacting particles as an example of a streaming kernel.  This time, we are going to advance things a bit and have particles that interact with each other and with the "media" in which they move.

Both gravitational and electrical forces are often modeled as deriving from a simple *potential*: the potential between two particles $p_1$ and $p_2$ is a function of their distance:

$$ \varphi(p_1, p_2) = k \frac{e_1 e_2}{\|r_1 - r_2\|}, $$

Where $r_1$ and $r_2$ are their positions, $e_1$ and $e_2$ are their charges, and $k$ is a scaling factor.  In this assignment we will assume $e_i = 1$ for all particles.  The force acting on $p_1$ due to the potential is the negative of its gradient with respect to $r_2$,

$$ F(p_1, p_2) = -\nabla_{r_1} \varphi(p_1, p_2). $$

The whole equal-and-opposite thing in physics implies that a force with the same magnitude and opposite direction acts on $p_2$: $F(p_2, p_1) = - F(p_1, p_2)$.  In this toy problem, all particle masses are equal to 1, so the acceleration due to a force is equivalent to the force.  The *total acceleration* experienced by a particle is the sum of the forces from all other particles:

$$ \partial v_i / \partial t = \sum_{j \neq i} F(p_i, p_j).$$

Now we see what makes this different from the streaming kernel we studied last week: to update one particle involves contributions from all other particles, $O(N_p^2)$ interactions!

Particles that are affected only by potentials is a good model for particles moving in a vaccuum, but sometimes we want to model particles moving in a medium, where the collide frequently and randomly with other particles.  This type of motion is called [Brownian motion](https://en.wikipedia.org/wiki/Brownian_motion). We'll skip a bunch of statistical physics and jump to the conclusion: whereas numerical time-stepping of a classical force
often takes the form of an update like

$$ x_{i,t+1} = x_{i,t} + f \Delta{t}, $$

Brownian motion looks like,

$$ x_{i,t+1} = x_{i,t} + \sqrt{2 d \Delta{t}}z, $$

where $z$ is a realization of a random variable.

If we have charged particles moving in a medium, then both potential and noisy contributions affect the motion of the particle.  Our program for this assignment includes both!  That makes this program more complicated to model and to optimize: we cannot reduce the performance down to the behavior of one kernel, but must try to evaluate when each kernel is the bottleneck.

Before you start with the actual assignment, let me show you what all of this looks like.
We run the program with `make runcloud`, like last time, and many of the variables that define the behavior of the target are the same: `NP` is the number of particles, `DT` is the step size, and `NT` is the number of steps. `K` is the potential coefficient $k$: negative values cause particles to attract and positive values cause particles to repel.  `D` is the diffusion coefficient of Brownian motion.

We can make this example look most like the last assignment by turning off Brownian motion and choosing negative $k$:

In [ ]:
make runcloud NP=32 DT=1.e-5 NT=400 D=0. K=-1.

You'll notice the output describes the "Hamiltonian" of the system.  This is like the total energy, and is theoretically conserved.  When I ran the above, it was conserved to 5 decimal places.  But these $n$-body systems are chaotic, and in particular they are unstable with particles that attract each other.  If we run the same program for just twice as long:

In [ ]:
make runcloud NP=32 DT=1.e-5 NT=800 D=0. K=-1.

For this assignment, I made a tool for us to visualize our simulations.  We can generate videos of our simulations, and we can embed them in this notebook.  We do this with the
`make vizcloud` target, which takes the same arguments as `make runcloud`, but also `CHUNK` (the number of time steps between frames of the video) and `VIZNAME`, the basename of the output video.  Like so:

In [ ]:
make vizcloud NP=8 DT=1.e-3 NT=800 D=0.0 K=-1. CHUNK=10 VIZNAME="attract"
display < attract.gif

You can see that the particles fling each other around, and with our fixed time step big errors occur when particles get close to each other.

You'll also notice that this problem occurs in a periodic domain, so particles can't get away from each other.  This ensures that they keep interacting, and is a common strategy to simulate an infinite, uniform distribution of particles.

If we instead model particles that repel each other, this behavior is a bit more stable:

In [ ]:
make vizcloud NP=8 DT=1.e-3 NT=800 D=0.0 K=1. CHUNK=10 VIZNAME="repel"
display < repel.gif

Finally, if we turn the potential interactions off, and turn the noisy interactions back on, we see browning motion:

In [ ]:
make vizcloud NP=8 DT=1.e-3 NT=800 D=0.1 K=0. CHUNK=10 VIZNAME="noisy"
display < noisy.gif

With Brownian motion, there is no "right" path for a particle to take, so any verification of the behavior has to be *statistical.*  In particular, one behavior of Brownian motion is that the distance a particle travels over time $\Delta{T}$ is on average $\sqrt{\Delta{T}}$.  We should be able to verify this: if we increase `NT` by a factor of 4, the average distance travelled should double.

In [ ]:
make runcloud NP=64 DT=1.e-3 NT=800 D=0.1 K=0.

In [ ]:
make runcloud NP=64 DT=1.e-3 NT=3200 D=0.1 K=0.

I know this is a long set up to the assignment, but our first project is going to keep building on this particle simulation, so I would like you to have a little understanding of what the parameters to the program mean.

For the remainder of this assignment, we'll keep the parameters `K=-1.`, `D=0.1`, and `DT=1.e-3` (which are the defaults in `Makefile`)

**Question 1 (5 pts)**: This program is currently a valid serial program.  Your first task is to convert it to a high-performing OpenMP program.  You will have to change

- The compiler flags (see the `OMPFLAGS` variable in `Makefile`)
- The runtime environment (You may either add new cells in this notebook that specify environment variables like `OMP_NUM_THREADS`, or you can add those in front of the targets, like `OMP_NUM_THREADS=5 make runcloud ...`)
- The code itself.  When modifying the code, you should show awareness of the issues that we discussed in class:
  - Scheduling: have you chosen a loop schedule that is balanced?  If your schedule is dynamic, is the additional overhead worth it?
  - Race conditions: how will you handle them?
  - Scoping: are you using the variables properly?

To modify the code, there are four operations that you will need to address.

- Initializing the variables, in `initialize.c`

In [ ]:
pygmentize initialize.c

- Compute the Hamiltonian, in `hamiltonian.c`

In [ ]:
pygmentize hamiltonian.c

- Updating the positions due to the velocities and the noise, in `verlet_stream_and_noise.c`

In [ ]:
pygmentize verlet_stream_and_noise.c

- Finally, we must compute the force due to potential interactions.  There are two different approaches to how we could go about doing this:
  1. Compute each interaction $F(p_i,p_j)$ twice: once for $p_i$, and once for $p_j$, then update them separately.
  2. Compute each interaction $F(p_i,p_j)$ once, and update both particles using the equal-and-opposite rule.
  
In this assignment, we are going to do our best to optimize both approaches.

One is in `verlet_accelerate.c`, the other is in `verlet_accelerate2.c`

In [ ]:
pygmentize verlet_accelerate.c

In [ ]:
pygmentize verlet_accelerate2.c # This gets used in runcloud2

Right now, there is a simple timer for the whole simulation.  Because this is a multikernel program, that may be insufficient to understand its behavior. 

We could modify the program to have timers for the individual kernels, or we could use profiling tools that lets us disentangle their contributions.

For this assignment, we will begin to use the tool [HPCToolkit](hpctoolkit.org) to profile our (parallel) applications.

The default class module should now load the various HPCToolkit commands.

In [ ]:
which hpcrun

Take a look at the [overview](http://hpctoolkit.org/man/hpctoolkit.html) for how we profile an application.  There are a few distinct steps.

First, we run the code with `hpcrun` to gather statistics.  Depending on how we call it, `hpcrun` can simply keep track of the walltime of different rountines, can generate traces for all threads (`hpcrun -t`), and can also do hardware event counters (including ones we've already seen with `perf`.

In [ ]:
hpcrun -L # Call this to get a great big list of hardware counters that are available.

When you call the program with `hpcrun`, a measurements directory should be created.

In [ ]:
make runcloud NP=64 DT=1.e-3 NT=800 D=0.1 K=0. PERF="hpcrun"

In [ ]:
ls -d hpctoolkit-*

Next, we have to help HPCToolkit know which measurements go with which part of the source code: we do this with `hpcstruct`:

In [ ]:
hpcstruct ./cloud

In [ ]:
ll cloud.hpcstruct

Finally we generate a profiling database with `hpcprof` by combining the `hpcstruct` file with the measurements:

In [ ]:
hpcprof -S cloud.hpcstruct hpctoolkit-cloud-measurements-*.ice-sched.pace.gatech.edu

Once you have the `hpctoolkit-database` directory, you can use the GUI tools `hpcviewer` and `hpctraceviewer` (if you used `hpcrun -t`), which will help you see walltime and/or hardware event counters for the routines in your program.

**Question 2 (5 pts)**: Call `make runcloud` and `make runcloud2` for different numbers of particles `NP`.  Try to scale `NT` so that each simulation runs for about a second.  Use HPCToolkit, get the data to generate a plot with three lines:

- $x$ axis: number of particles.
- $y$ axis: walltime per particle time step (so take walltime and divide by `NP * NT`
- Line 1: the time taken by `verlet_step_stream_and_noise` in either version of the simulation
- Line 2: the time taken by `verlet_accelerate` in `runcloud`
- Line 3: the time taken by `verlet_accelerate2` in `runcloud2`